In [1]:


import pandas as pd

In [3]:
df = pd.read_csv('classification.csv')

In [4]:
zero_count = df['Purchased'].value_counts().get(0, 0)


In [5]:
print("Occurrences of zero:", zero_count)


Occurrences of zero: 257


In [6]:

def calculate_gini_index(dataframe, target_column):
    total_samples = len(dataframe)
    
    # Step 1: Calculate the proportion of each class
    class_proportions = dataframe[target_column].value_counts() / total_samples
    
    # Step 2: Calculate the squared proportion of each class
    squared_proportions = class_proportions ** 2
    
    # Step 3: Sum up the squared proportions
    gini_index = 1 - squared_proportions.sum()
    
    return gini_index

# Replace 'Purchased' with the actual column name in your dataset
gini_index = calculate_gini_index(df, 'Purchased')

print("Gini Index:", gini_index)

Gini Index: 0.45938750000000006


In [8]:
import pandas as pd

def calculate_gini_index_by_category(dataframe, feature1, feature2, target_column):
    total_samples = len(dataframe)
    gini_index_total = 0.0
    
    # Loop through unique values in feature1
    for value1 in dataframe[feature1].unique():
        subset_feature1 = dataframe[dataframe[feature1] == value1]
        
        # Loop through unique values in feature2 within the subset of feature1
        for value2 in subset_feature1[feature2].unique():
            subset_feature2 = subset_feature1[subset_feature1[feature2] == value2]
            
            # Calculate Gini index for the subset
            gini_index_subset = calculate_gini_index(subset_feature2, target_column)
            
            # Weight the Gini index by the proportion of samples in the subset
            weight = len(subset_feature2) / total_samples
            gini_index_total += weight * gini_index_subset
    
    return gini_index_total

def calculate_gini_index(dataframe, target_column):
    total_samples = len(dataframe)
    
    # Calculate the Gini index for the entire dataset
    class_proportions = dataframe[target_column].value_counts() / total_samples
    squared_proportions = class_proportions ** 2
    gini_index = 1 - squared_proportions.sum()
    
    return gini_index

# Replace 'Age', 'EstimatedSalary', and 'Purchased' with the actual column names in your dataset
gini_index = calculate_gini_index_by_category(df, 'Age', 'EstimatedSalary', 'Purchased')

print("Gini Index for Age and EstimatedSalary categories:", gini_index)



Gini Index for Age and EstimatedSalary categories: 0.005


In [12]:
import pandas as pd
import numpy as np

class DecisionTree:
    def __init__(self):
        self.tree = None

    def calculate_gini(self, labels):
        unique_labels, counts = np.unique(labels, return_counts=True)
        probabilities = counts / len(labels)
        gini = 1 - np.sum(probabilities**2)
        return gini

    def calculate_information_gain(self, data, feature, target):
        gini_parent = self.calculate_gini(data[target])

        unique_values = data[feature].unique()
        weighted_gini_child = 0

        for value in unique_values:
            subset = data[data[feature] == value]
            weight = len(subset) / len(data)
            gini_child = self.calculate_gini(subset[target])
            weighted_gini_child += weight * gini_child

        information_gain = gini_parent - weighted_gini_child
        return information_gain

    def find_best_split(self, data, target):
        features = data.columns[:-1]  # Exclude the target column
        best_feature = None
        best_information_gain = -1

        for feature in features:
            information_gain = self.calculate_information_gain(data, feature, target)

            if information_gain > best_information_gain:
                best_feature = feature
                best_information_gain = information_gain

        return best_feature

    def build_tree(self, data, target):
        unique_labels = data[target].unique()

        if len(unique_labels) == 1:
            # If all instances have the same label, create a leaf node
            return {'label': unique_labels[0]}

        if len(data.columns) == 1:
            # If there are no more features to split on, create a leaf node with the majority label
            majority_label = data[target].mode().iloc[0]
            return {'label': majority_label}

        # Find the best feature to split on
        best_feature = self.find_best_split(data, target)

        # Split the data based on the best feature
        unique_values = data[best_feature].unique()
        sub_trees = {}
        for value in unique_values:
            subset = data[data[best_feature] == value]
            sub_trees[value] = self.build_tree(subset.drop(columns=[best_feature]), target)

        return {'feature': best_feature, 'sub_trees': sub_trees}

    def fit(self, data, target):
        self.tree = self.build_tree(data, target)

    def predict_instance(self, instance, tree):
        if 'label' in tree:
            return tree['label']
        else:
            feature_value = instance[tree['feature']]
            if feature_value in tree['sub_trees']:
                return self.predict_instance(instance, tree['sub_trees'][feature_value])
            else:
                # If the feature value is not present in the training data, return a default prediction
                return list(tree['sub_trees'].values())[0]['label']

    def predict_proba_instance(self, instance, tree):
        if 'label' in tree:
            return {tree['label']: 1.0}
        else:
            feature_value = instance[tree['feature']]
            if feature_value in tree['sub_trees']:
                return self.predict_proba_instance(instance, tree['sub_trees'][feature_value])
            else:
                # If the feature value is not present in the training data, return a default prediction
                return self.predict_proba_instance(instance, list(tree['sub_trees'].values())[0])

    def predict(self, data):
        predictions = []
        for _, instance in data.iterrows():
            predictions.append(self.predict_instance(instance, self.tree))
        return predictions

    def predict_proba(self, data):
        probabilities = []
        for _, instance in data.iterrows():
            probabilities.append(self.predict_proba_instance(instance, self.tree))
        return probabilities

# Example usage:
# Creating a sample dataset
data = pd.DataFrame({
    'Age': ['Young', 'Young', 'Young', 'Middle-aged', 'Middle-aged', 'Middle-aged', 'Senior', 'Senior', 'Senior'],
    'Salary': ['Low', 'Low', 'High', 'Low', 'Low', 'High', 'Low', 'Low', 'High'],
    'Purchased': [0, 0, 1, 0, 1, 1, 1, 0, 1]
})

# Creating an instance of the DecisionTree class
tree_model = DecisionTree()

# Fitting the model
tree_model.fit(data, target='Purchased')

# Creating a new data instance for prediction
new_data_instance = pd.DataFrame({'Age': ['Young'], 'Salary': ['Low']})

# Making predictions
prediction = tree_model.predict(new_data_instance)
probabilities = tree_model.predict_proba(new_data_instance)

print("Predicted Output:", prediction)
print("Predicted Probabilities:", probabilities)



Predicted Output: [0]
Predicted Probabilities: [{0: 1.0}]
